In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Stock Bull - Exploratory Data Analysis\n",
    "\n",
    "This notebook explores the training data and provides insights for model development."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "\n",
    "from src.data_preparation.data_loader import DataLoader\n",
    "from src.utils.data_analyzer import DataAnalyzer\n",
    "\n",
    "# Set style\n",
    "sns.set_style('whitegrid')\n",
    "plt.rcParams['figure.figsize'] = (12, 6)\n",
    "\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load data\n",
    "loader = DataLoader()\n",
    "df = loader.load_training_data()\n",
    "\n",
    "# Calculate returns and labels\n",
    "df = loader.calculate_future_returns(df, horizon_days=30)\n",
    "df = loader.create_labels(df)\n",
    "\n",
    "print(f\"Dataset shape: {df.shape}\")\n",
    "print(f\"Date range: {df['date'].min()} to {df['date'].max()}\")\n",
    "print(f\"Number of stocks: {df['symbol'].nunique()}\")\n",
    "\n",
    "df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Class Distribution"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze class balance\n",
    "analyzer = DataAnalyzer(df)\n",
    "analyzer.analyze_class_balance()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Feature Correlations"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Top correlated features\n",
    "correlations = analyzer.analyze_feature_correlations(top_n=20)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Temporal Trends"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze trends over time\n",
    "analyzer.analyze_temporal_trends()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Missing Values"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Check missing values\n",
    "analyzer.analyze_missing_values()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Stock-wise Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Analyze individual stock\n",
    "stock_symbol = 'RELIANCE'\n",
    "stock_df = df[df['symbol'] == stock_symbol].copy()\n",
    "\n",
    "# Plot price and predictions\n",
    "fig, axes = plt.subplots(2, 1, figsize=(14, 10))\n",
    "\n",
    "# Price chart\n",
    "axes[0].plot(stock_df['date'], stock_df['close'])\n",
    "axes[0].set_title(f'{stock_symbol} - Price History')\n",
    "axes[0].set_ylabel('Close Price')\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# Future returns distribution\n",
    "axes[1].hist(stock_df['future_return'].dropna(), bins=50, edgecolor='black')\n",
    "axes[1].set_title(f'{stock_symbol} - Future Returns Distribution')\n",
    "axes[1].set_xlabel('30-day Future Return (%)')\n",
    "axes[1].set_ylabel('Frequency')\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Feature Statistics"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get numeric columns\n",
    "numeric_cols = df.select_dtypes(include=[np.number]).columns\n",
    "feature_cols = [col for col in numeric_cols if col not in ['label', 'future_return']]\n",
    "\n",
    "# Summary statistics\n",
    "df[feature_cols].describe().T"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. Sector Analysis"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Load stock metadata\n",
    "from data-pipeline.storage.database import DatabaseManager, Stock\n",
    "\n",
    "db = DatabaseManager()\n",
    "session = db.get_session()\n",
    "\n",
    "stocks_df = pd.read_sql(session.query(Stock).statement, session.bind)\n",
    "\n",
    "# Merge with main dataframe\n",
    "df_with_sector = df.merge(stocks_df[['symbol', 'sector']], on='symbol', how='left')\n",
    "\n",
    "# Sector distribution\n",
    "sector_dist = df_with_sector.groupby(['sector', 'label_name']).size().unstack(fill_value=0)\n",
    "\n",
    "sector_dist.plot(kind='bar', stacked=True, figsize=(14, 6))\n",
    "plt.title('Label Distribution by Sector')\n",
    "plt.xlabel('Sector')\n",
    "plt.ylabel('Count')\n",
    "plt.legend(title='Label')\n",
    "plt.xticks(rotation=45, ha='right')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Conclusion\n",
    "\n",
    "Key findings:\n",
    "1. Class distribution shows imbalance (address with SMOTE or class weights)\n",
    "2. Top correlated features identified for feature selection\n",
    "3. Some features have missing values (handle with forward fill)\n",
    "4. Temporal patterns visible in data\n",
    "5. Sector-specific patterns may benefit from specialized models"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}